# Bi Data
värden samlade från den digitala bikupan som jag tar hand om upp vid I11.
informationsmängden består av 
* lufttemperatur
* lufttryck
* luftfuktighet
* Datum
* Vikt
  Målet är att prediktera vikt ändringen vid ett specifikt väderförhållande

### Load the data
Datat är hämtat från grafana instansen som jag har tillgång till för att kunna visualizera insamlade värden.
Det är sendan importerat i en sql databas där det är normalizerat så att medelvärden är uttagna per timme.

In [47]:
import pandas as pd
url = "Bi-Data3.csv"
##columns = ['Time','Temp','Luftfuktighet','Lufttryck','Vikt','ViktSkillnad']
df = pd.read_csv(url, delimiter=';')
df.head()


Year               int64
Month              int64
Day                int64
Hour               int64
Temp             float64
Luftfuktighet    float64
Lutftryck        float64
Vikt             float64
ViktSkillnad     float64
dtype: object


In [49]:
#some info of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4051 entries, 0 to 4050
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           4051 non-null   int64  
 1   Month          4051 non-null   int64  
 2   Day            4051 non-null   int64  
 3   Hour           4051 non-null   int64  
 4   Temp           4051 non-null   float64
 5   Luftfuktighet  4051 non-null   float64
 6   Lutftryck      4051 non-null   float64
 7   Vikt           4051 non-null   float64
 8   ViktSkillnad   4051 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 285.0 KB


### Visualize the data

In [ ]:
# Visualize the whole dataset
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df.corr(),annot=True)
plt.show()

In [67]:
#target vector
Y = df["ViktSkillnad"]
#Feature
X = df[["Year","Month","Day","Hour","Temp","Luftfuktighet","Lutftryck","Vikt"]]

### Split data


In [68]:
from sklearn.model_selection import train_test_split
# Split the data to train and test dataset.
from random import random
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)


### Evaluate_models

In [69]:
#Random forest
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

forestModel = make_pipeline(
    SelectKBest(k="all"),
    StandardScaler(),
    RandomForestRegressor(n_estimators=100,
                          max_depth=50,
                          random_state=77,
                          n_jobs=-1
                        )
)
forestModel.fit(X_train,y_train)


Pipeline(steps=[('selectkbest', SelectKBest(k='all')),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=50, n_jobs=-1,
                                       random_state=77))])

In [54]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lm = make_pipeline(
    StandardScaler(),
    LinearRegression(),
)
lm.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [70]:
#"Evaluate"
from sklearn.metrics import mean_absolute_error

print('Linear Regression training MAE:', round(mean_absolute_error(y_train, lm.predict(X_train)),5))
print('Linear Regression Test MAE:', round(mean_absolute_error(y_test, lm.predict(X_test)),5))

print('Random forest regressor model training MAE:', round(mean_absolute_error(y_train, forestModel.predict(X_train)),5))
print('Random forest regressor model Test MAE:', round(mean_absolute_error(y_test, forestModel.predict(X_test)),5))

Linear Regression training MAE: 0.96105
Linear Regression Test MAE: 0.95814
Random forest regressor model training MAE: 0.13395
Random forest regressor model Test MAE: 0.3479


In [71]:
# Make predictions on validation dataset
from sklearn.metrics import accuracy_score
predictions = lm.predict(X_test)
# Evaluate predictions
LMaccuracy = lm.score(X_test, y_test)
print ('Linear Regression model prediction accuracy:' , LMaccuracy)
FMaccuracy = forestModel.score(X_test, y_test)
print ('Random forest regressor model prediction accuracy:' , FMaccuracy)


Linear Regression model prediction accuracy: -0.12552643317621648
Random forest regressor model prediction accuracy: -0.22204997155633643


### Test the model

In [74]:
#Test
#Year Month Day Hour Temp Luftfuktighet Lutftryck Vikt
import numpy as np
X_new = np.array([[2022,7,4,8,16.702363,45.479137,928.238448,120.394482],[2023,7,4,8,25,40,950,110]]) #[0.250149],[60.495833]
forestModel.predict(X_new) 

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(


array([0.47269524, 0.43482609])